In [1]:
# %tensorflow_version 2.x
# !pip uninstall -y tensorflow
# !pip install tensorflow-gpu==1.15.0

In [2]:
import tensorflow as tf
import numpy as np
import scipy.io
import timeit
import sys
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import random
import seaborn as sns
import pandas as pd
from glob import glob 
import xlwt
import os 

In [3]:
class neural_net(object):
    def __init__(self, *inputs, layers, monotonic = False, normalize = False, LAA = False):

        self.layers = layers
        self.num_layers = len(self.layers)
        self.monotonic = monotonic
        self.normalize = normalize
        self.LAA = LAA
        
        if self.normalize:
            if len(inputs) == 0:
                in_dim = self.layers[0]
                self.X_mean = np.zeros([1, in_dim])
                self.X_std = np.ones([1, in_dim])
            else:
                X = np.concatenate(inputs, 1)
                self.X_mean = X.mean(0, keepdims=True)
                self.X_std = X.std(0, keepdims=True)

        self.weights = []
        self.biases = []
        self.A = []

        for l in range(0,self.num_layers-1):
            in_dim = self.layers[l]
            out_dim = self.layers[l+1]
            xavier_stddev = np.sqrt(2/(in_dim + out_dim))
            W = tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev = xavier_stddev),dtype=tf.float32, trainable=True) 
            # monotonically increasing neural network
            if self.monotonic:
                W = W**2
            b = tf.Variable(np.zeros([1, out_dim]), dtype=tf.float32, trainable=True)
            
            # tensorflow variables
            self.weights.append(W)
            self.biases.append(b)
            
            # locally adaptive activation
            if self.LAA:
                a = tf.Variable(0.05, dtype=tf.float32)
                self.A.append(a)
            
    def __call__(self, *inputs):
        if self.normalize:
            H = (tf.concat(inputs, 1) - self.X_mean)/self.X_std
        else:
            H = tf.concat(inputs, 1)

        for l in range(0, self.num_layers-1):
            W = self.weights[l]
            b = self.biases[l]
            
            # matrix multiplication
            H = tf.matmul(H, W)
            # add bias
            H = H + b
            # activation
            if l < self.num_layers-2:
                if self.LAA:
                    H = tf.tanh(20 * self.A[l]*H)
                else:
                    H = tf.tanh(H)

        return H

In [4]:
def tf_session():
    # tf session
    config = tf.ConfigProto(allow_soft_placement=True,  # an operation might be placed on CPU for example if GPU is not available
                            log_device_placement=True)  # print out device information
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    # init
    init = tf.global_variables_initializer()
    sess.run(init)

    return sess

In [5]:
def Gardner(psi, theta_r, theta_s, alpha, K_s):
    K = K_s * tf.exp(alpha * psi)
    theta = theta_r + (theta_s - theta_r) * tf.exp(alpha * psi)
    return theta, K

In [6]:
# theta_r, theta_s, alpha [/cm], K_s [cm/hour]
# layer 1 is the lower layer while layer 2 is the upper layer
Srivastava_parameters = {
            "layer1": [0.06, 0.40, 1.0, 1.0],
            "layer2": [0.06, 0.40, 1.0, 10.0]}

In [7]:
# sess = tf_session()

# log10_h = np.arange(-2, 6, 0.1)  # logarithm of suction (negative matric potential) in cm
# psi = -10**log10_h

# tf_psi = tf.constant(psi, dtype = tf.float32)

# tf_theta_1, tf_K_1 = Gardner(tf_psi, *Srivastava_parameters["layer1"])
# tf_theta_2, tf_K_2 = Gardner(tf_psi, *Srivastava_parameters["layer2"])
# psi = sess.run(tf_psi)
# theta_1 = sess.run(tf_theta_1)
# theta_2 = sess.run(tf_theta_2)
# K_1 = sess.run(tf_K_1)
# K_2 = sess.run(tf_K_2)

In [8]:
# plt.plot(np.log10(-psi), theta_1, label = "layer1")
# plt.plot(np.log10(-psi), theta_2, label =  "layer2")
# plt.title("Water Retention Curve")
# plt.legend()
# plt.xlabel("log$_{10}(-\\psi)$")
# plt.ylabel("$\\theta$")

In [9]:
# plt.plot(np.log10(-psi), K_1, label = "layer 1")
# plt.plot(np.log10(-psi), K_2, label = "layer 2")
# plt.title("Hydraulic Conductivity Function")
# plt.legend()
# plt.xlabel("log$_{10}(-\\psi)$")
# plt.ylabel("$K [cm / hour]$")

In [10]:
def log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

In [11]:
def RRE_mixed_1D(psi, theta, K, t, z):
    
    theta_t = tf.gradients(theta, t)[0]
    theta_z = tf.gradients(theta, z)[0]
    
    psi_z = tf.gradients(psi, z)[0]
    psi_zz = tf.gradients(psi_z, z)[0]
    
    K_z = tf.gradients(K, z)[0]

    qz = -K*(psi_z + 1.0)
    residual = theta_t - K_z*psi_z - K*psi_zz - K_z
    return residual, qz

In [12]:
class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name = None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name
    def sample(self, N):
        x = self.coords[0:1,:] + (self.coords[1:2,:]-self.coords[0:1,:])*np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y

In [13]:
class PINNs_RRE1D(object):
    """
    PINNs_RRE1D class implements physics-informed neural network solver for 
    Ricahrdson-Ricahrds equation for a two-layered soil using TensorFlow 1.15.
    # notational conventions
    _tf: placeholders
    _pred: output of neural networks or other parametric models
    _res: values related to residual points
    _data: training data 
    _ic: initial condition
    _bc: boundary condition (ub: upper; lb: lower for 1D)
    _star: prediction (used after training)
    _1: lower layer
    _2: upper layer
    _int: interface of the layers
    t: time
    z: vertical coordinate
    theta: volumetric water content
    psi: matric potential
    K: hydraulic conductivity
    ALR: adaptive learning rate
    LAA: layer-wise locally adaptive activation functions
    weight_values: [ic_1, lb, res_2, ic_2, ub, res_int, psi_int, qz_int]
    """
    def __init__(self, t_res_1, z_res_1,
                       t_res_2, z_res_2,
                       t_int, z_int,
                       t_ic_1, z_ic_1, theta_ic_1,
                       t_ic_2, z_ic_2, theta_ic_2,
                       t_ub, z_ub, qz_ub,
                       t_lb, z_lb, theta_lb,
                       layers_psi_1, layers_psi_2, hydraulic_model, 
                       hydraulic_parameters_1, hydraulic_parameters_2, weight_values,
                       normalize, ALR, LAA):
        
        # NN architecture
        self.layers_psi_1 = layers_psi_1
        self.layers_psi_2 = layers_psi_2
        
        # Mode 
        self.hydraulic_model = hydraulic_model  # "monotonic", "VGM", "Brooks","PDI", "Gardner"
        self.normalize = normalize # normalize input values to neural networks
        self.ALR = ALR  # adaptive learning rate
        self.LAA = LAA  # locally adaptive activation functions
        
        # Adaptive re-weighting constant
        self.rate = 0.9
        
        # parameter for the output of neural networks
        self.beta = 1.0
        
        # initial values for weights in the loss function
        
        self.adaptive_constant_ic_1_val = np.array(weight_values[0])
        self.adaptive_constant_lb_val = np.array(weight_values[1])
        
        self.adaptive_constant_res_2_val = np.array(weight_values[2])
        self.adaptive_constant_ic_2_val = np.array(weight_values[3])
        self.adaptive_constant_ub_val = np.array(weight_values[4])
        
        self.adaptive_constant_res_int_val = np.array(weight_values[5])
        self.adaptive_constant_psi_int_val = np.array(weight_values[6])
        self.adaptive_constant_qz_int_val = np.array(weight_values[7])
        
        # hydraulic parameters  
                    
        if self.hydraulic_model == "Gardner":  # theta_r, theta_s, alpha, K_s
            self.theta_r_1 = tf.Variable(hydraulic_parameters_1[0], dtype=tf.float32, trainable=False)
            self.theta_s_1 = tf.Variable(hydraulic_parameters_1[1], dtype=tf.float32, trainable=False)
            self.alpha_1 = tf.Variable(hydraulic_parameters_1[2], dtype=tf.float32, trainable=False)
            self.K_s_1 = tf.Variable(hydraulic_parameters_1[3], dtype=tf.float32, trainable=False)
            
            self.theta_r_2 = tf.Variable(hydraulic_parameters_2[0], dtype=tf.float32, trainable=False)
            self.theta_s_2 = tf.Variable(hydraulic_parameters_2[1], dtype=tf.float32, trainable=False)
            self.alpha_2 = tf.Variable(hydraulic_parameters_2[2], dtype=tf.float32, trainable=False)
            self.K_s_2 = tf.Variable(hydraulic_parameters_2[3], dtype=tf.float32, trainable=False)

        # data
        
        [self.t_res_1, self.z_res_1] = [t_res_1, z_res_1]
        [self.t_res_2, self.z_res_2] = [t_res_2, z_res_2]
        [self.t_int, self.z_int] = [t_int, z_int]
        [self.t_ic_1, self.z_ic_1, self.theta_ic_1] = [t_ic_1, z_ic_1, theta_ic_1]
        [self.t_ic_2, self.z_ic_2, self.theta_ic_2] = [t_ic_2, z_ic_2, theta_ic_2]
        [self.t_ub, self.z_ub, self.qz_ub] = [t_ub, z_ub, qz_ub]
        [self.t_lb, self.z_lb, self.theta_lb] = [t_lb, z_lb, theta_lb]
        
        # placeholders
        
        [self.t_res_1_tf, self.z_res_1_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(2)]
        [self.t_res_2_tf, self.z_res_2_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(2)]
        [self.t_int_tf, self.z_int_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(2)]

        [self.t_ic_1_tf, self.z_ic_1_tf, self.theta_ic_1_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(3)]
        [self.t_ic_2_tf, self.z_ic_2_tf, self.theta_ic_2_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(3)]

        [self.t_ub_tf, self.z_ub_tf, self.qz_ub_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(3)]
        [self.t_lb_tf, self.z_lb_tf, self.theta_lb_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(3)]
        
        
        # weight parameters in the loss function
        self.adaptive_constant_ic_1_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_ic_1_val.shape)
        self.adaptive_constant_lb_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_lb_val.shape)
        
        self.adaptive_constant_res_2_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_res_2_val.shape)
        self.adaptive_constant_ic_2_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_ic_2_val.shape)
        self.adaptive_constant_ub_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_ub_val.shape)
        self.adaptive_constant_res_int_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_res_int_val.shape)
        self.adaptive_constant_psi_int_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_psi_int_val.shape)
        self.adaptive_constant_qz_int_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_qz_int_val.shape)
        
        # neural network definition
        self.net_psi_1 = neural_net(self.t_res_1, self.z_res_1, layers = self.layers_psi_1, monotonic = False, normalize = self.normalize, LAA = self.LAA)
        self.net_psi_2 = neural_net(self.t_res_2, self.z_res_2, layers = self.layers_psi_2, monotonic = False, normalize = self.normalize, LAA = self.LAA)
     
        # prediction at initial condition
        
        self.log_psi_ic_1_pred = self.net_psi_1(self.t_ic_1_tf, self.z_ic_1_tf)
        self.psi_ic_1_pred = -tf.exp(self.log_psi_ic_1_pred) + self.beta
        self.theta_ic_1_pred, self.K_ic_1_pred = Gardner(self.psi_ic_1_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.K_s_1)

        self.log_psi_ic_2_pred = self.net_psi_2(self.t_ic_2_tf, self.z_ic_2_tf)
        self.psi_ic_2_pred = -tf.exp(self.log_psi_ic_2_pred) + self.beta
        self.theta_ic_2_pred, self.K_ic_2_pred = Gardner(self.psi_ic_2_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.K_s_2)
        
        # prediction at upper boundary
        self.log_psi_ub_pred = self.net_psi_2(self.t_ub_tf, self.z_ub_tf)
        self.psi_ub_pred = -tf.exp(self.log_psi_ub_pred) + self.beta
        self.theta_ub_pred, self.K_ub_pred = Gardner(self.psi_ub_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.K_s_2)
        
        self.qz_ub_pred = RRE_mixed_1D(self.psi_ub_pred, self.theta_ub_pred, self.K_ub_pred,
                                                                self.t_ub_tf, self.z_ub_tf)[1]
            
           
        # prediction at lower boundary
        self.log_psi_lb_pred = self.net_psi_1(self.t_lb_tf, self.z_lb_tf)
        self.psi_lb_pred = -tf.exp(self.log_psi_lb_pred) + self.beta
        self.theta_lb_pred, self.K_lb_pred = Gardner(self.psi_lb_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.K_s_1)
        
        # prediction at residual points
        self.log_psi_res_1_pred = self.net_psi_1(self.t_res_1_tf, self.z_res_1_tf)
        self.psi_res_1_pred = -tf.exp(self.log_psi_res_1_pred) + self.beta
        self.theta_res_1_pred, self.K_res_1_pred = Gardner(self.psi_res_1_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.K_s_1)
              
        self.residual_res_1_pred, self.qz_res_1_pred = RRE_mixed_1D(self.psi_res_1_pred, self.theta_res_1_pred, self.K_res_1_pred,
                                                                self.t_res_1_tf, self.z_res_1_tf)
        
        self.log_psi_res_2_pred = self.net_psi_2(self.t_res_2_tf, self.z_res_2_tf)
        self.psi_res_2_pred = -tf.exp(self.log_psi_res_2_pred) + self.beta
        self.theta_res_2_pred, self.K_res_2_pred = Gardner(self.psi_res_2_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.K_s_2)
              
        self.residual_res_2_pred, self.qz_res_2_pred = RRE_mixed_1D(self.psi_res_2_pred, self.theta_res_2_pred, self.K_res_2_pred,
                                                                self.t_res_2_tf, self.z_res_2_tf)

        # psi, flux, and residual at the layer interface
        # layer 1
        self.log_psi_int_1_pred = self.net_psi_1(self.t_int_tf, self.z_int_tf)
        self.psi_int_1_pred = -tf.exp(self.log_psi_int_1_pred) + self.beta
        self.theta_int_1_pred, self.K_int_1_pred = Gardner(self.psi_int_1_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.K_s_1)
              
        self.residual_int_1_pred, self.qz_int_1_pred = RRE_mixed_1D(self.psi_int_1_pred, self.theta_int_1_pred, self.K_int_1_pred,
                                                                self.t_int_tf, self.z_int_tf)
        
        # layer 2
        self.log_psi_int_2_pred = self.net_psi_2(self.t_int_tf, self.z_int_tf)
        self.psi_int_2_pred = -tf.exp(self.log_psi_int_2_pred) + self.beta
        self.theta_int_2_pred, self.K_int_2_pred = Gardner(self.psi_int_2_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.K_s_2)
              
        self.residual_int_2_pred, self.qz_int_2_pred = RRE_mixed_1D(self.psi_int_2_pred, self.theta_int_2_pred, self.K_int_2_pred,
                                                                self.t_int_tf, self.z_int_tf)
                
        # loss
        # layer 1
        self.loss_res_1 =  tf.reduce_mean(tf.square(self.residual_res_1_pred))
        self.loss_ic_1 = tf.reduce_mean(tf.square(self.theta_ic_1_pred - self.theta_ic_1_tf))
        self.loss_lb = tf.reduce_mean(tf.square(self.theta_lb_pred - self.theta_lb_tf))
        
        self.loss_1 = self.loss_res_1 +  self.adaptive_constant_ic_1_tf * self.loss_ic_1  \
                                      +  self.adaptive_constant_lb_tf * self.loss_lb
        
        # layer 2
        self.loss_res_2 =  tf.reduce_mean(tf.square(self.residual_res_2_pred))
        self.loss_ic_2 = tf.reduce_mean(tf.square(self.theta_ic_2_pred - self.theta_ic_2_tf))
        self.loss_ub = tf.reduce_mean(tf.square(self.qz_ub_pred - self.qz_ub_tf))
        
        self.loss_2 = self.adaptive_constant_res_2_tf * self.loss_res_2 +  self.adaptive_constant_ic_2_tf * self.loss_ic_2  \
                                      +  self.adaptive_constant_ub_tf * self.loss_ub
        
        self.loss_res_int = tf.reduce_mean(tf.square(self.residual_int_1_pred - self.residual_int_2_pred))
        self.loss_psi_int = tf.reduce_mean(tf.square(self.log_psi_int_1_pred - self.log_psi_int_2_pred))
        self.loss_qz_int = tf.reduce_mean(tf.square(self.qz_int_1_pred - self.qz_int_2_pred))
        
        self.loss_int = self.adaptive_constant_res_int_tf * self.loss_res_int  \
                         +  self.adaptive_constant_psi_int_tf * self.loss_psi_int  \
                         +  self.adaptive_constant_qz_int_tf * self.loss_qz_int
        
        
        self.loss = self.loss_1 + self.loss_2 + self.loss_int
        
                    
        # Define optimizer with learning rate schedule
        self.global_step = tf.Variable(0, trainable = False)
        starter_learning_rate = 1e-3
        self.learning_rate = tf.train.exponential_decay(starter_learning_rate, self.global_step,
                                                        1000, 0.90, staircase=False)
        
        # Passing global_step to minimize() will increment it at each step.
        # self.train_op_1 = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss_1, global_step=self.global_step)
        # self.train_op_2 = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss_2, global_step=self.global_step)
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)
        

        # L-BFGS-B method
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                        method = 'L-BFGS-B',
                                                        options = {'maxiter': 50000,
                                                                   'maxfun': 50000,
                                                                   'maxcor': 50,
                                                                   'maxls': 50,
                                                                   'ftol' : 1e-10,
                                                                   'gtol' : 1e-8})
        
        
        # Logger
        
        self.loss_res_1_log = []
        self.loss_ic_1_log = []
        self.loss_lb_log = []
        self.loss_res_2_log = []
        self.loss_ic_2_log = []
        self.loss_ub_log = []
        self.loss_psi_int_log = []
        self.loss_qz_int_log = []
        self.loss_res_int_log = []
        
        self.loss_res_1_log_BFGS = []
        self.loss_ic_1_log_BFGS = []
        self.loss_lb_log_BFGS = []
        self.loss_res_2_log_BFGS = []
        self.loss_ic_2_log_BFGS = []
        self.loss_ub_log_BFGS = []
        self.loss_psi_int_log_BFGS = []
        self.loss_qz_int_log_BFGS = []
        self.loss_res_int_log_BFGS = []
        
        self.adaptive_activation_1 = []
        self.adaptive_activation_2 = []
        
        self.saver = tf.train.Saver()
        
        # new residual points
        self.new_residual_1 = []
        self.new_residual_2 = []
        
        # Store the adaptive constant
        
        self.adaptive_constant_ic_1_log = []
        self.adaptive_constant_lb_log = []
        
        self.adaptive_constant_res_2_log = []
        self.adaptive_constant_ic_2_log = []
        self.adaptive_constant_ub_log = []
        
        self.adaptive_constant_res_int_log = []
        self.adaptive_constant_psi_int_log = []
        self.adaptive_constant_qz_int_log = []
        
                
        # Compute the adaptive constant
        
        self.adaptive_constant_ic_1_list = []
        self.adaptive_constant_lb_list = []
        
        self.adaptive_constant_res_2_list = []
        self.adaptive_constant_ic_2_list = []
        self.adaptive_constant_ub_list = []
        
        self.adaptive_constant_res_int_list = []
        self.adaptive_constant_psi_int_list = []
        self.adaptive_constant_qz_int_list = []
        
        
        self.sess = tf_session()
    
    def train_Adam(self, iterations, batch = True, batch_size = 128):

        N_res_1 = self.t_res_1.shape[0]
        N_res_2 = self.t_res_2.shape[0]

        start_time = timeit.default_timer()
        running_time = 0
        it = 0
        for it in range(iterations):
           
            if batch:

                idx_res_1 = np.random.choice(N_res_1, batch_size, replace = False)
                idx_res_2 = np.random.choice(N_res_2, batch_size, replace = False)

                (t_res_1, z_res_1) = (self.t_res_1[idx_res_1,:],
                                  self.z_res_1[idx_res_1,:])
                (t_res_2, z_res_2) = (self.t_res_2[idx_res_2,:],
                                  self.z_res_2[idx_res_2,:])
                
            else:

                (t_res_1, z_res_1) = (self.t_res_1,
                                  self.z_res_1)
                (t_res_2, z_res_2) = (self.t_res_2,
                                  self.z_res_2)

            tf_dict = {self.t_res_1_tf: t_res_1,
                       self.z_res_1_tf: z_res_1,
                       self.t_res_2_tf: t_res_2,
                       self.z_res_2_tf: z_res_2,
                       self.t_int_tf: self.t_int,
                       self.z_int_tf: self.z_int,
                       self.t_ic_1_tf: self.t_ic_1,
                       self.z_ic_1_tf: self.z_ic_1,
                       self.theta_ic_1_tf: self.theta_ic_1,
                       self.t_ic_2_tf: self.t_ic_2,
                       self.z_ic_2_tf: self.z_ic_2,
                       self.theta_ic_2_tf: self.theta_ic_2,
                       self.t_ub_tf: self.t_ub,
                       self.z_ub_tf: self.z_ub,
                       self.qz_ub_tf: self.qz_ub,
                       self.t_lb_tf: self.t_lb,
                       self.z_lb_tf: self.z_lb,
                       self.theta_lb_tf: self.theta_lb,
                       self.adaptive_constant_ic_1_tf: self.adaptive_constant_ic_1_val,
                       self.adaptive_constant_lb_tf: self.adaptive_constant_lb_val,
                       self.adaptive_constant_res_2_tf: self.adaptive_constant_res_2_val,
                       self.adaptive_constant_ic_2_tf: self.adaptive_constant_ic_2_val,
                       self.adaptive_constant_ub_tf: self.adaptive_constant_ub_val, 
                       self.adaptive_constant_res_int_tf: self.adaptive_constant_res_int_val,
                       self.adaptive_constant_psi_int_tf: self.adaptive_constant_psi_int_val,
                       self.adaptive_constant_qz_int_tf: self.adaptive_constant_qz_int_val}
        
        
#             self.sess.run(self.train_op_1, tf_dict)
#             self.sess.run(self.train_op_2, tf_dict)
            self.sess.run(self.train_op, tf_dict)
    
            # Print
            if it % 10 == 0:
                elapsed = timeit.default_timer() - start_time
                running_time += elapsed/3600.0
                
                
        
        
                loss_res_1_value, loss_ic_1_value, loss_lb_value = self.sess.run([self.loss_res_1, self.loss_ic_1, self.loss_lb], tf_dict)
                loss_res_2_value, loss_ic_2_value, loss_ub_value = self.sess.run([self.loss_res_2, self.loss_ic_2, self.loss_ub], tf_dict)

                loss_psi_int_value, loss_qz_int_value, loss_res_int_value = self.sess.run([self.loss_psi_int, self.loss_qz_int, self.loss_res_int], tf_dict)
                
                adaptive_activation_1 = self.sess.run(self.net_psi_1.A, tf_dict)
                adaptive_activation_2 = self.sess.run(self.net_psi_2.A, tf_dict)
                
                self.loss_res_1_log.append(loss_res_1_value)
                self.loss_ic_1_log.append(loss_ic_1_value)
                self.loss_lb_log.append(loss_lb_value)
                self.loss_res_2_log.append(loss_res_2_value)
                self.loss_ic_2_log.append(loss_ic_2_value)
                self.loss_ub_log.append(loss_ub_value)
                self.loss_psi_int_log.append(loss_psi_int_value)
                self.loss_qz_int_log.append(loss_qz_int_value)
                self.loss_res_int_log.append(loss_res_int_value)   
                
                self.adaptive_activation_1.append(adaptive_activation_1)
                self.adaptive_activation_2.append(adaptive_activation_2)
                
                print('It: %d, Loss_res_1: %.3e, Loss_ic_1: %.3e, Loss_lb: %.3e,  \
                               Loss_res_2: %.3e, Loss_ic_2: %.3e, Loss_ub: %.3e,  \
                               Loss_psi_int: %.3e, Loss_qz_int: %.3e, Loss_res_int: %.3e, Time: %.2f' %
                      (it, loss_res_1_value, loss_ic_1_value, loss_lb_value,
                           loss_res_2_value, loss_ic_2_value, loss_ub_value,
                           loss_psi_int_value, loss_qz_int_value, loss_res_int_value, elapsed))
                
                start_time = timeit.default_timer()
                                               
    
    def train_BFGS(self):
        # full batch L-BFGS
        tf_dict = {self.t_res_1_tf: self.t_res_1,
                       self.z_res_1_tf: self.z_res_1,
                       self.t_res_2_tf: self.t_res_2,
                       self.z_res_2_tf: self.z_res_2,
                       self.t_int_tf: self.t_int,
                       self.z_int_tf: self.z_int,
                       self.t_ic_1_tf: self.t_ic_1,
                       self.z_ic_1_tf: self.z_ic_1,
                       self.theta_ic_1_tf: self.theta_ic_1,
                       self.t_ic_2_tf: self.t_ic_2,
                       self.z_ic_2_tf: self.z_ic_2,
                       self.theta_ic_2_tf: self.theta_ic_2,
                       self.t_ub_tf: self.t_ub,
                       self.z_ub_tf: self.z_ub,
                       self.qz_ub_tf: self.qz_ub,
                       self.t_lb_tf: self.t_lb,
                       self.z_lb_tf: self.z_lb,
                       self.theta_lb_tf: self.theta_lb,
                       self.adaptive_constant_ic_1_tf: self.adaptive_constant_ic_1_val,
                       self.adaptive_constant_lb_tf: self.adaptive_constant_lb_val,
                       self.adaptive_constant_res_2_tf: self.adaptive_constant_res_2_val,
                       self.adaptive_constant_ic_2_tf: self.adaptive_constant_ic_2_val,
                       self.adaptive_constant_ub_tf: self.adaptive_constant_ub_val, 
                       self.adaptive_constant_res_int_tf: self.adaptive_constant_res_int_val,
                       self.adaptive_constant_psi_int_tf: self.adaptive_constant_psi_int_val,
                       self.adaptive_constant_qz_int_tf: self.adaptive_constant_qz_int_val}
        
#         L-BFGS-B
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss_res_1, self.loss_ic_1, self.loss_lb,
                                           self.loss_res_2, self.loss_ic_2, self.loss_ub,
                                           self.loss_psi_int, self.loss_qz_int, self.loss_res_int],
                                loss_callback = self.callback)
        
    def callback(self, loss_res_1_value, loss_ic_1_value, loss_lb_value,
                           loss_res_2_value, loss_ic_2_value, loss_ub_value,
                           loss_psi_int_value, loss_qz_int_value, loss_res_int_value):
        
        tf_dict = {self.t_res_1_tf: self.t_res_1,
                       self.z_res_1_tf: self.z_res_1,
                       self.t_res_2_tf: self.t_res_2,
                       self.z_res_2_tf: self.z_res_2,
                       self.t_int_tf: self.t_int,
                       self.z_int_tf: self.z_int,
                       self.t_ic_1_tf: self.t_ic_1,
                       self.z_ic_1_tf: self.z_ic_1,
                       self.theta_ic_1_tf: self.theta_ic_1,
                       self.t_ic_2_tf: self.t_ic_2,
                       self.z_ic_2_tf: self.z_ic_2,
                       self.theta_ic_2_tf: self.theta_ic_2,
                       self.t_ub_tf: self.t_ub,
                       self.z_ub_tf: self.z_ub,
                       self.qz_ub_tf: self.qz_ub,
                       self.t_lb_tf: self.t_lb,
                       self.z_lb_tf: self.z_lb,
                       self.theta_lb_tf: self.theta_lb,
                       self.adaptive_constant_ic_1_tf: self.adaptive_constant_ic_1_val,
                       self.adaptive_constant_lb_tf: self.adaptive_constant_lb_val,
                       self.adaptive_constant_res_2_tf: self.adaptive_constant_res_2_val,
                       self.adaptive_constant_ic_2_tf: self.adaptive_constant_ic_2_val,
                       self.adaptive_constant_ub_tf: self.adaptive_constant_ub_val, 
                       self.adaptive_constant_res_int_tf: self.adaptive_constant_res_int_val,
                       self.adaptive_constant_psi_int_tf: self.adaptive_constant_psi_int_val,
                       self.adaptive_constant_qz_int_tf: self.adaptive_constant_qz_int_val}
        
        adaptive_activation_1 = self.sess.run(self.net_psi_1.A, tf_dict)
        adaptive_activation_2 = self.sess.run(self.net_psi_2.A, tf_dict)
        
        self.adaptive_activation_1.append(adaptive_activation_1)
        self.adaptive_activation_2.append(adaptive_activation_2)
            
        
        self.loss_res_1_log_BFGS.append(loss_res_1_value)
        self.loss_ic_1_log_BFGS.append(loss_ic_1_value)
        self.loss_lb_log_BFGS.append(loss_lb_value)
        self.loss_res_2_log_BFGS.append(loss_res_2_value)
        self.loss_ic_2_log_BFGS.append(loss_ic_2_value)
        self.loss_ub_log_BFGS.append(loss_ub_value)
        self.loss_psi_int_log_BFGS.append(loss_psi_int_value)
        self.loss_qz_int_log_BFGS.append(loss_qz_int_value)
        self.loss_res_int_log_BFGS.append(loss_res_int_value)   
                
                
        print('Loss_res_1: %.3e, Loss_ic_1: %.3e, Loss_lb: %.3e,  \n\
                               Loss_res_2: %.3e, Loss_ic_2: %.3e, Loss_ub: %.3e,  \n\
                               Loss_psi_int: %.3e, Loss_qz_int: %.3e, Loss_res_int: %.3e' %
                      (loss_res_1_value, loss_ic_1_value, loss_lb_value,
                           loss_res_2_value, loss_ic_2_value, loss_ub_value,
                           loss_psi_int_value, loss_qz_int_value, loss_res_int_value))
        
        
    def RAR(self, dom_coords_1, dom_coords_2, iterations, batchsize, m):
        self.train_Adam(iterations, batchsize)
        
        i = 1
        
        while i < 10:
            res_sampler_1 = Sampler(2, dom_coords_1, lambda x: 0)
            res_sampler_2 = Sampler(2, dom_coords_2, lambda x: 0)
            Z_res_1, _ = res_sampler_1.sample(np.int32(1e6))
            Z_res_2, _ = res_sampler_2.sample(np.int32(1e6))
            z_res_1, t_res_1 = Z_res_1[:, 0:1], Z_res_1[:, 1:2]
            z_res_2, t_res_2 = Z_res_2[:, 0:1], Z_res_2[:, 1:2]
        
            tf_dict = {self.t_res_1_tf: t_res_1,
                       self.z_res_1_tf: z_res_1,
                       self.t_res_2_tf: t_res_2,
                       self.z_res_2_tf: z_res_2}
            residual_1 = self.sess.run(self.residual_res_1_pred, tf_dict)
            residual_2 = self.sess.run(self.residual_res_2_pred, tf_dict)

            err_abs_1 = np.abs(residual_1)
            err_abs_2 = np.abs(residual_2)
            err_1 = np.mean(err_abs_1)
            err_2 = np.mean(err_abs_2)
            print("Mean residual for layer 1: %.3e, Mean residual for layer 2: %.3e" % (err_1, err_2))

            # choose m largest residual
            z_id_1 = err_abs_1.argsort(axis = 0)[-m * i:][::-1][:,0]
            z_id_2 = err_abs_2.argsort(axis = 0)[-m * i:][::-1][:,0]

            print(z_id_1, z_id_2)
            print("Adding new point for layer 1:", Z_res_1[z_id_1, :], "\n")
            print("High residuals for layer 1 are:", err_abs_1[z_id_1], "\n")
            print("Adding new point for layer 2:", Z_res_2[z_id_2, :], "\n")
            print("High residuals for layer 2 are:", err_abs_2[z_id_2], "\n")
        
            self.new_residual_1.append(Z_res_1[z_id_1, :])
            self.new_residual_2.append(Z_res_2[z_id_2, :])
            
            self.z_res_1 = np.vstack((self.z_res_1, Z_res_1[z_id_1, 0:1]))
            self.t_res_1 = np.vstack((self.t_res_1, Z_res_1[z_id_1, 1:2]))
            
            self.z_res_2 = np.vstack((self.z_res_2, Z_res_2[z_id_2, 0:1]))
            self.t_res_2 = np.vstack((self.t_res_2, Z_res_2[z_id_2, 1:2]))
            
            self.train_Adam(iterations, batchsize)
            
            i += 1

            print(i)
            
    def predict_1(self, t_star, z_star):

        tf_dict = {self.t_res_1_tf: t_star, self.z_res_1_tf: z_star}

        psi_star = self.sess.run(self.psi_res_1_pred, tf_dict)
        theta_star = self.sess.run(self.theta_res_1_pred, tf_dict)
        K_star = self.sess.run(self.K_res_1_pred, tf_dict)
        qz_star = self.sess.run(self.qz_res_1_pred, tf_dict)
        
        residual = self.sess.run(self.residual_res_1_pred, tf_dict)

        return psi_star, theta_star, K_star, qz_star, residual
    
    def predict_2(self, t_star, z_star):

        tf_dict = {self.t_res_2_tf: t_star, self.z_res_2_tf: z_star}

        psi_star = self.sess.run(self.psi_res_2_pred, tf_dict)
        theta_star = self.sess.run(self.theta_res_2_pred, tf_dict)
        K_star = self.sess.run(self.K_res_2_pred, tf_dict)
        qz_star = self.sess.run(self.qz_res_2_pred, tf_dict)
        
        residual = self.sess.run(self.residual_res_2_pred, tf_dict)

        return psi_star, theta_star, K_star, qz_star, residual

In [14]:
# from google.colab import drive
# drive.mount("/content/drive")

In [15]:
# cd drive/My\ Drive

In [16]:
# cd UFnets/forward_heterogeneous/

In [17]:
# dom_coords_1 = np.array([[0.0, 0.0],   # t = 0 hour to t = 10 hours
#                        [10.0, -10.0]])
# dom_coords_2 = np.array([[0.0, 10.0],   # t = 0 hour to t = 10 hours
#                        [10.0, 0.0]])
# int_coords = np.array([[0.0, 0.0],
#                        [10.0, 0.0]])

In [18]:
# produce residual points 
# res_sampler_1 = Sampler(2, dom_coords_1, lambda x: 0)
# res_sampler_2 = Sampler(2, dom_coords_2, lambda x: 0)
# int_sampler = Sampler(2, int_coords, lambda x: 0)
# np.random.seed(0)
# Z_res_1, _ = res_sampler_1.sample(np.int32(100000))
# Z_res_2, _ = res_sampler_2.sample(np.int32(100000))
# Z_int, _ = int_sampler.sample(np.int32(10000))
# np.save("data/residual_points_layer1", Z_res_1)
# np.save("data/residual_points_layer2", Z_res_2)
# np.save("data/residual_points_interface", Z_int)

In [19]:
def main_loop(folder, ID, random_seed, iterations, number_int = 1000, LAA = True, ALR = False, weight_values = [10, 10, 10, 10, 10, 10, 10, 10],
              number_layers_1 = 5, number_units_1 = 50, number_layers_2 = 5, number_units_2 = 50, 
              number_res_1 = 10000, number_res_2 = 10000, number_ub = 1000,
              RAR = False, batch_size = 128, new_residual_points = 10):
    """
    folder: the name of the folder to save
    ALR: adaptive learning rate
    LAA: layer-wise locally adaptive activation functions
    RAR: residual adaptive refinement
    weight_values: [ic_1, lb, res_2, ic_2, ub, res_int, psi_int, qz_int]
    """
    
    if not os.path.isdir(f'../results/{folder}'):
        os.mkdir(f'../results/{folder}')

    if not os.path.isdir(f'../results/{folder}/{ID}'):
        os.mkdir(f'../results/{folder}/{ID}')

    # domain boundaries; the first column is time, the second column is depth (positive upward)
    ics_coords_1 = np.array([[0.0, 0.0],
                       [0.0, -10.0]])
    ics_coords_2 = np.array([[0.0, 10.0],
                           [0.0, 0.0]])
    lb_coords = np.array([[0.0, -10.0],   # upper boundary condition
                           [10.0, -10.0]])
    ub_coords = np.array([[0.0, 10.0],   # lower boundary condition
                           [10.0, 10.0]])
    dom_coords_1 = np.array([[0.0, 0.0],   # t = 0 hour to t = 10 hours
                           [10.0, -10.0]])
    dom_coords_2 = np.array([[0.0, 10.0],   # t = 0 hour to t = 10 hours
                           [10.0, 0.0]])
    int_coords = np.array([[0.0, 0.0],
                           [10.0, 0.0]])

    # import true solution: first index is depth, second index is time
    psi_true = np.load(f"../analytical_solutions/Srivastava_psi_heterogeneous.npy")

    # import residual points 
    Z_res_1 = np.load(f"data/residual_points_layer1.npy")
    Z_res_2 = np.load(f"data/residual_points_layer2.npy")
    Z_int = np.load(f"data/residual_points_interface.npy")

    # hydraulic paramters
    hydraulic_parameters_1 = Srivastava_parameters["layer1"]
    hydraulic_parameters_2 = Srivastava_parameters["layer2"]


    # hydralic function model
    hydraulic_model = "Gardner"

    # normalization
    normalize = False

    # NN architecture
    layers_psi_1 = [2] + number_layers_1*[number_units_1] + [1]
    layers_psi_2 = [2] + number_layers_2*[number_units_2] + [1]

    # forward solution
    # collocation points
    if RAR:
        number_res_1_2 =  number_res_1 - (RAR_iterations - 1) * 10
        number_res_2_2 =  number_res_2 - (RAR_iterations - 1) * 10
    else:
        number_res_1_2 = number_res_1
        number_res_2_2 = number_res_2


    t_res_1 = Z_res_1[:number_res_1_2, 0:1]
    z_res_1 = Z_res_1[:number_res_1_2, 1:2]

    t_res_2 = Z_res_2[:number_res_2_2, 0:1]
    z_res_2 = Z_res_2[:number_res_2_2, 1:2]

    t_res = np.concatenate((t_res_2, t_res_1), axis = 1)
    z_res = np.concatenate((z_res_2, z_res_1), axis = 1) 

    t_int = Z_int[:number_int, 0:1]
    z_int = Z_int[:number_int, 1:2]

    # initial points

    t_ic_1 = np.linspace(ics_coords_1[0, 0], ics_coords_1[1, 0], 101)[:, None]
    z_ic_1 = np.linspace(ics_coords_1[0, 1], ics_coords_1[1, 1], 101)[:, None]

    t_ic_2 = np.linspace(ics_coords_2[0, 0], ics_coords_2[1, 0], 101)[:, None]
    z_ic_2 = np.linspace(ics_coords_2[0, 1], ics_coords_2[1, 1], 101)[:, None]


    # upper boundary points

    t_ub = np.linspace(ub_coords[0, 0], ub_coords[1, 0], number_ub + 1)[1:, None]
    z_ub = np.linspace(ub_coords[0, 1], ub_coords[1, 1], number_ub + 1)[1:, None]

    # lower boundary points

    t_lb = np.linspace(lb_coords[0, 0], lb_coords[1, 0], 101)[:, None]
    z_lb = np.linspace(lb_coords[0, 1], lb_coords[1, 1], 101)[:, None]

    # initial condition
    # upper water flux condition (positive downward) [cm/hour]
    q_A1_star = 0.1
    q_A2_star = 0.1
    q_B2_star = 0.9

    # parameters
    alpha1 = 1.0
    alpha2 = 1.0
    K_s1 = 1.0
    K_s2 = 10.0
    theta_s1 = 0.40
    theta_s2 = 0.40
    theta_r1 = 0.06
    theta_r2 = 0.06
    L1_star = 10.0
    L2_star = 10.0

    L1 = L1_star * alpha1
    L2 = L2_star * alpha2

    # pressure at the lower boundary (water table)
    psi_0 = 0.0

    q_A1 = q_A1_star / K_s1

    q_A2 = q_A2_star / K_s2

    q_B2 = q_B2_star / K_s2

    # initial condition
    K_ic_1 = q_A1 - (q_A1 - np.exp(alpha1 * psi_0)) * np.exp(-(L1 + z_ic_1 * alpha1))
    psi_ic_1 = np.log(K_ic_1) /alpha1

    K_ic_2 = q_A2 - (q_A2 - (q_A1 - (q_A1 - np.exp(alpha1 * psi_0)) \
           *np.exp(-L1))**(alpha2/alpha1))* np.exp(-z_ic_2 * alpha2)
    psi_ic_2 = np.log(K_ic_2) /alpha2

    theta_ic_1 = theta_r1 + (theta_s1 - theta_r1) * np.exp(alpha1 * psi_ic_1)
    theta_ic_2 = theta_r2 + (theta_s2 - theta_r2) * np.exp(alpha2 * psi_ic_2)


    # boundary conditions
    theta_lb = theta_r1 + (theta_s1 - theta_r1) * np.exp(alpha1 * psi_0)
    qz_ub = -q_B2_star * np.ones(t_ub.shape)
    theta_lb = theta_lb * np.ones(t_lb.shape)

    # initialization
    tf.reset_default_graph()
    tf.set_random_seed(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)

    # training    
    model = PINNs_RRE1D(
                t_res_1, z_res_1,
                t_res_2, z_res_2,
                t_int, z_int,
                t_ic_1, z_ic_1, theta_ic_1,
                t_ic_2, z_ic_2, theta_ic_2,
                t_ub, z_ub, qz_ub,
                t_lb, z_lb, theta_lb,      
                layers_psi_1, layers_psi_2, hydraulic_model, hydraulic_parameters_1, hydraulic_parameters_2,
                weight_values, normalize, ALR, LAA
                   )

    # Adam training
    start_time = timeit.default_timer() 

    if RAR:
        iterations = iterations//10
        model.RAR(dom_coords, iterations, batch_size, new_residual_points)
    else:
        model.train_Adam(iterations, batch_size)
    time_Adam = timeit.default_timer() - start_time

    # BFGS training
    start_time = timeit.default_timer() 
    model.train_BFGS()
    time_BFGS = timeit.default_timer() - start_time

    # evaluation
    T = 101
    N = 101
    t_1 = np.linspace(dom_coords_1[0, 0], dom_coords_1[1, 0], T)[:, None]
    z_1 = np.linspace(dom_coords_1[0, 1], dom_coords_1[1, 1], N)[:, None]
    z_1_mesh, t_1_mesh = np.meshgrid(z_1, t_1)
    X_star_1 = np.hstack((t_1_mesh.flatten()[:, None], z_1_mesh.flatten()[:, None]))

    t_2 = np.linspace(dom_coords_2[0, 0], dom_coords_2[1, 0], T)[:, None]
    z_2 = np.linspace(dom_coords_2[0, 1], dom_coords_2[1, 1], N)[:, None]
    z_2_mesh, t_2_mesh = np.meshgrid(z_2, t_2)
    X_star_2 = np.hstack((t_2_mesh.flatten()[:, None], z_2_mesh.flatten()[:, None]))

    t_test_1 = X_star_1[:, 0:1]
    z_test_1 = X_star_1[:, 1:2]

    t_test_2 = X_star_2[:, 0:1]
    z_test_2 = X_star_2[:, 1:2]

    # prediction
    psi_1_pred, theta_1_pred, K_1_pred, flux_1_pred, residual_1 = model.predict_1(t_test_1, z_test_1)
    psi_2_pred, theta_2_pred, K_2_pred, flux_2_pred, residual_2 = model.predict_2(t_test_2, z_test_2)

    theta_1_pred = theta_1_pred.reshape((T, N))
    K_1_pred = K_1_pred.reshape((T, N))
    psi_1_pred = psi_1_pred.reshape((T, N))
    flux_1_pred = flux_1_pred.reshape((T, N))
    residual_1 = residual_1.reshape((T, N))


    theta_2_pred = theta_2_pred.reshape((T, N))
    K_2_pred = K_2_pred.reshape((T, N))
    psi_2_pred = psi_2_pred.reshape((T, N))
    flux_2_pred = flux_2_pred.reshape((T, N))
    residual_2 = residual_2.reshape((T, N))

    psi_pred = np.concatenate((psi_2_pred, psi_1_pred), axis = 1) 
    theta_pred = np.concatenate((theta_2_pred, theta_1_pred), axis = 1)
    K_pred = np.concatenate((K_2_pred, K_1_pred), axis = 1)
    residual = np.concatenate((residual_2, residual_1), axis = 1)
    flux_pred = np.concatenate((flux_2_pred, flux_1_pred), axis = 1) 

    t_mesh = np.concatenate((t_2_mesh, t_1_mesh), axis = 1)
    z_mesh = np.concatenate((z_2_mesh, z_1_mesh), axis = 1) 

    theta_1_true = theta_r1 + (theta_s1 - theta_r1) * np.exp(alpha1 * psi_true[101:,:])
    theta_2_true = theta_r2 + (theta_s2 - theta_r2) * np.exp(alpha2 * psi_true[0:101,:])

    theta_true = np.concatenate((theta_2_true, theta_1_true), axis = 0)

    # the error at the interface is computed "twice" (theta in layer 1 and layer 2, which are not necesarily the same)
    theta_L2_error = np.sqrt(np.mean((theta_true.T - theta_pred) ** 2))
    theta_abs_error = np.mean(np.abs(theta_true.T- theta_pred))
    theta_square = np.sqrt(np.mean((theta_true.T) ** 2))
    theta_max_error = np.max(np.abs(theta_true.T- theta_pred))

    theta_L2_relative_error = theta_L2_error/theta_square
    theta_abs_relative_error = theta_abs_error/np.mean(np.abs(theta_true.T))

    print("theta_L2_relative_error is", theta_L2_relative_error)
    print("theta_abs_relative_error is", theta_abs_relative_error)
    print("theta_max_error is", theta_max_error)

    psi_L2_error = np.sqrt(np.mean((psi_true.T - psi_pred) ** 2))
    psi_abs_error = np.mean(np.abs(psi_true.T- psi_pred))
    psi_square = np.sqrt(np.mean((psi_true.T) ** 2))
    psi_max_error = np.max(np.abs(psi_true.T- psi_pred))

    psi_L2_relative_error = psi_L2_error/psi_square
    psi_abs_relative_error = psi_abs_error/np.mean(np.abs(psi_true.T))


    print("psi_L2_relative_error is", psi_L2_relative_error)
    print("psi_abs_relative_error is", psi_abs_relative_error)
    print("psi_max_error is", psi_max_error)

    # theta distribution
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    c1 = ax[0].pcolor(t_mesh, z_mesh - 10.0, theta_pred, cmap='gnuplot2_r')
    fig.colorbar(c1, ax = ax[0])
    ax[0].set_xlabel(r'$t$ [hours]')
    ax[0].set_ylabel(r'$z$ [cm]')
    ax[0].set_title('Predicted $\\theta$')

    c2 = ax[1].pcolor(t_mesh, z_mesh - 10.0, theta_true.T, cmap='gnuplot2_r')
    fig.colorbar(c2, ax = ax[1])
    ax[1].set_xlabel(r'$t$ [hours]')
    ax[1].set_title('True $\\theta$')

    c3 = ax[2].pcolor(t_mesh, z_mesh - 10.0, np.abs(theta_pred - theta_true.T), cmap='gnuplot2_r')
    fig.colorbar(c3, ax = ax[2])
    ax[2].set_xlabel(r'$t$ [hours]')
    ax[2].set_title('Absolute error in $\\theta$')

    fig.savefig(f'../results/{folder}/{ID}/theta_color.png')
    fig.clf()

    # psi distribution
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    c1 = ax[0].pcolor(t_mesh, z_mesh - 10.0, psi_pred, cmap='gnuplot2_r')
    fig.colorbar(c1, ax = ax[0])
    ax[0].set_xlabel(r'$t$ [hours]')
    ax[0].set_ylabel(r'$z$ [cm]')
    ax[0].set_title('Predicted $\\psi$ [cm]')

    c2 = ax[1].pcolor(t_mesh, z_mesh - 10.0, psi_true.T, cmap='gnuplot2_r')
    fig.colorbar(c2, ax = ax[1])
    ax[1].set_xlabel(r'$t$ [hours]')
    ax[1].set_title('True $\\psi$ [cm]')

    c3 = ax[2].pcolor(t_mesh, z_mesh - 10.0, np.abs(psi_pred - psi_true.T), cmap='gnuplot2_r')
    fig.colorbar(c3, ax = ax[2])
    ax[2].set_xlabel(r'$t$ [hours]')
    ax[2].set_title('Absolute error in $\\psi$ [cm]')

    fig.savefig(f'../results/{folder}/{ID}/psi_color.png')
    fig.clf()

    fig, ax = plt.subplots(1, 1, figsize=(6.5, 6.5))

    c1 = ax.pcolor(t_mesh, z_mesh - 10.0, flux_pred, cmap='gnuplot2_r')
    fig.colorbar(c1, ax = ax)
    ax.set_xlabel(r'$t$ [hours]')
    ax.set_ylabel(r'$z$ [cm]')
    ax.set_title('Predicted $q$ [cm/hours]')

    fig.savefig(f'../results/{folder}/{ID}/flux.png')
    fig.clf()

    fig = plt.subplots(1, 1, figsize=(6.5, 6.5))

    plt.scatter(theta_pred[0, ::5], z_mesh[0, ::5] - 10.0, s = 15, label = 'Predicted')
    plt.scatter(theta_pred[1, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[5, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[10, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[30, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[50, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[100, ::5], z_mesh[0, ::5] - 10.0, s = 15)

    plt.plot(theta_true.T[0, :], z_mesh[0, :] - 10.0 ,label = 't = 0 hour')
    plt.plot(theta_true.T[1, :], z_mesh[0, :] - 10.0,label = 't = 0.1 hour')
    plt.plot(theta_true.T[5, :], z_mesh[0, :] - 10.0,label = 't = 0.5 hour')
    plt.plot(theta_true.T[10, :], z_mesh[0, :]- 10.0 ,label = 't = 1 hour')
    plt.plot(theta_true.T[30, :], z_mesh[0, :]- 10.0,label = 't = 3 hour')
    plt.plot(theta_true.T[50, :], z_mesh[0, :]- 10.0,label = 't = 5 hour')
    plt.plot(theta_true.T[100, :], z_mesh[0, :]- 10.0, label = 't = 10 hour')

    plt.xlim(0.05,0.50)
    plt.xlabel('$\\theta$')
    plt.ylabel('$z$ cm')
    plt.legend(loc = (0.75,0.55), fontsize = 8)

    plt.savefig(f'../results/{folder}/{ID}/theta_fixed_time.png')
    plt.clf()

    t_res = np.concatenate((t_res_2, t_res_1), axis = 1)
    z_res = np.concatenate((z_res_2, z_res_1), axis = 1) 

    fig = plt.figure(1, figsize=(6.5, 6.5))

    plt.plot(t_res, z_res - 10.0, 'rx', markersize = 3, label = "residual points")

    if RAR:
        new_residual = model.new_residual[0]
        for i in range(1, len(model.new_residual)):
            new_residual = np.vstack((new_residual, model.new_residual[i]))
        plt.plot(new_residual[:, 0], new_residual[:, 1] - 10.0, 'bx', markersize = 5, label = "added residual points")



    plt.xlabel(r'$t$ [hours]')
    plt.ylabel(r'$z$ [cm]')
    plt.title('residual points')
    plt.xlim(-0.1, 10.1)
    plt.ylim(-10.1, 0.1)
    plt.legend(loc = (0.55,0.15))
    plt.savefig(f'../results/{folder}/{ID}/residual_points.png', transparent = True)
    plt.clf()

    fig = plt.figure(1, figsize=(6.5, 6.5))

    plt.pcolor(t_mesh, z_mesh - 10.0, np.log10(np.abs(residual)), cmap='gnuplot2_r')

    plt.colorbar()
    plt.xlabel(r'$t$ [hours]')
    plt.ylabel(r'$z$ [cm]')
    plt.title('$log_{10}r(x)$')
    plt.savefig(f'../results/{folder}/{ID}/residual.png', transparent = True)
    plt.clf()

    # Loss

    loss_res_1 = model.loss_res_1_log
    loss_ic_1 = model.loss_ic_1_log
    loss_lb = model.loss_lb_log
    loss_res_2 = model.loss_res_2_log
    loss_ic_2 = model.loss_ic_2_log
    loss_ub = model.loss_ub_log
    loss_psi_int = model.loss_psi_int_log
    loss_qz_int = model.loss_qz_int_log
    loss_res_int = model.loss_res_int_log

    Adam_iterations = np.arange(0, len(loss_res_1)) * 10

    loss_res_1_BFGS = model.loss_res_1_log_BFGS
    loss_ic_1_BFGS = model.loss_ic_1_log_BFGS
    loss_lb_BFGS = model.loss_lb_log_BFGS
    loss_res_2_BFGS = model.loss_res_2_log_BFGS
    loss_ic_2_BFGS = model.loss_ic_2_log_BFGS
    loss_ub_BFGS = model.loss_ub_log_BFGS
    loss_psi_int_BFGS = model.loss_psi_int_log_BFGS
    loss_qz_int_BFGS = model.loss_qz_int_log_BFGS
    loss_res_int_BFGS = model.loss_res_int_log_BFGS

    BFGS_iterations = np.arange(0, len(loss_res_1_BFGS))

    total_iterations = np.append(Adam_iterations, BFGS_iterations + Adam_iterations[-1])

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    # Adam
    ax[0].plot(Adam_iterations, loss_ic_1, label='$\mathcal{L}_{ic1}$')
    ax[0].plot(Adam_iterations, loss_ic_2, label='$\mathcal{L}_{ic2}$')
    ax[0].plot(Adam_iterations, loss_ub ,label='$\mathcal{L}_{ub}$')
    ax[0].plot(Adam_iterations, loss_lb,label='$\mathcal{L}_{lb}$')
    ax[0].plot(Adam_iterations, loss_res_1, label='$\mathcal{L}_{res1}$')
    ax[0].plot(Adam_iterations, loss_res_2, label='$\mathcal{L}_{res2}$')
    ax[0].plot(Adam_iterations, loss_psi_int, label='$\mathcal{L}_{\psi-int}$')
    ax[0].plot(Adam_iterations, loss_qz_int, label='$\mathcal{L}_{qz-int}$')
    ax[0].plot(Adam_iterations, loss_res_int, label='$\mathcal{L}_{res-int}$')
    ax[0].set_yscale('log')
    ax[0].set_xlabel('iterations')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Adam')
    ax[0].legend()

    # BFGS
    ax[1].plot(BFGS_iterations, loss_ic_1_BFGS, label='$\mathcal{L}_{ic1}$')
    ax[1].plot(BFGS_iterations, loss_ic_2_BFGS, label='$\mathcal{L}_{ic2}$')
    ax[1].plot(BFGS_iterations, loss_ub_BFGS ,label='$\mathcal{L}_{ub}$')
    ax[1].plot(BFGS_iterations, loss_lb_BFGS,label='$\mathcal{L}_{lb}$')
    ax[1].plot(BFGS_iterations, loss_res_1_BFGS, label='$\mathcal{L}_{res1}$')
    ax[1].plot(BFGS_iterations, loss_res_2_BFGS, label='$\mathcal{L}_{res2}$')
    ax[1].plot(BFGS_iterations, loss_psi_int_BFGS, label='$\mathcal{L}_{\psi-int}$')
    ax[1].plot(BFGS_iterations, loss_qz_int_BFGS, label='$\mathcal{L}_{qz-int}$')
    ax[1].plot(BFGS_iterations, loss_res_int_BFGS, label='$\mathcal{L}_{res-int}$')

    ax[1].set_yscale('log')
    ax[1].set_xlabel('iterations')
    ax[1].set_ylabel('Loss')
    ax[1].set_title('L-BFGS-B')
    ax[1].legend()


    # total training
    ax[2].plot(total_iterations, loss_ic_1 + loss_ic_1_BFGS, label='$\mathcal{L}_{ic1}$')
    ax[2].plot(total_iterations, loss_ic_2 + loss_ic_2_BFGS, label='$\mathcal{L}_{ic2}$')
    ax[2].plot(total_iterations, loss_ub + loss_ub_BFGS ,label='$\mathcal{L}_{ub}$')
    ax[2].plot(total_iterations, loss_lb + loss_lb_BFGS,label='$\mathcal{L}_{lb}$')
    ax[2].plot(total_iterations, loss_res_1 + loss_res_1_BFGS, label='$\mathcal{L}_{res1}$')
    ax[2].plot(total_iterations, loss_res_2 + loss_res_2_BFGS, label='$\mathcal{L}_{res2}$')
    ax[2].plot(total_iterations, loss_psi_int + loss_psi_int_BFGS, label='$\mathcal{L}_{\psi-int}$')
    ax[2].plot(total_iterations, loss_qz_int + loss_qz_int_BFGS, label='$\mathcal{L}_{qz-int}$')
    ax[2].plot(total_iterations, loss_res_int + loss_res_int_BFGS, label='$\mathcal{L}_{res-int}$')


    ax[2].set_yscale('log')
    ax[2].set_xlabel('iterations')
    ax[2].set_ylabel('Loss')
    ax[2].set_title('total')
    ax[2].legend()

    plt.savefig(f'../results/{folder}/{ID}/training.png')
    plt.clf()

    # layer-wise locallyadaptive parameters for NN1 (lower)
    adaptive_parameter_layer1 = []
    adaptive_parameter_layer2 = []
    adaptive_parameter_layer3 = []
    adaptive_parameter_layer4 = []
    adaptive_parameter_layer5 = []
    for i in range(len(model.adaptive_activation_1)):
        adaptive_parameters = model.adaptive_activation_1[i]
        adaptive_parameter_layer1.append(adaptive_parameters[0]* 20)
        adaptive_parameter_layer2.append(adaptive_parameters[1]* 20)
        adaptive_parameter_layer3.append(adaptive_parameters[2]* 20)
        adaptive_parameter_layer4.append(adaptive_parameters[3]* 20)
        adaptive_parameter_layer5.append(adaptive_parameters[4]* 20)

    plt.plot(adaptive_parameter_layer1, label = 'layer1')
    plt.plot(adaptive_parameter_layer2, label = 'layer2')
    plt.plot(adaptive_parameter_layer3, label = 'layer3')
    plt.plot(adaptive_parameter_layer4, label = 'layer4')
    plt.plot(adaptive_parameter_layer5, label = 'layer5')
    plt.legend()
    plt.ylabel('sa')
    plt.xlabel('iteration')

    plt.savefig(f'../results/{folder}/{ID}/adaptive_activation_parameter_NN1.png', transparent = True)
    plt.clf()
    
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer1_1", adaptive_parameter_layer1)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer2_1", adaptive_parameter_layer2)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer3_1", adaptive_parameter_layer3)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer4_1", adaptive_parameter_layer4)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer5_1", adaptive_parameter_layer5)
    
    # layer-wise locally adaptive parameters for NN2 (upper)
    
    adaptive_parameter_layer1 = []
    adaptive_parameter_layer2 = []
    adaptive_parameter_layer3 = []
    adaptive_parameter_layer4 = []
    adaptive_parameter_layer5 = []
    for i in range(len(model.adaptive_activation_2)):
        adaptive_parameters = model.adaptive_activation_2[i]
        adaptive_parameter_layer1.append(adaptive_parameters[0]* 20)
        adaptive_parameter_layer2.append(adaptive_parameters[1]* 20)
        adaptive_parameter_layer3.append(adaptive_parameters[2]* 20)
        adaptive_parameter_layer4.append(adaptive_parameters[3]* 20)
        adaptive_parameter_layer5.append(adaptive_parameters[4]* 20)

    plt.plot(adaptive_parameter_layer1, label = 'layer1')
    plt.plot(adaptive_parameter_layer2, label = 'layer2')
    plt.plot(adaptive_parameter_layer3, label = 'layer3')
    plt.plot(adaptive_parameter_layer4, label = 'layer4')
    plt.plot(adaptive_parameter_layer5, label = 'layer5')
    plt.legend()
    plt.ylabel('sa')
    plt.xlabel('iteration')

    plt.savefig(f'../results/{folder}/{ID}/adaptive_activation_parameter_NN2.png', transparent = True)
    plt.clf()
    
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer1_2", adaptive_parameter_layer1)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer2_2", adaptive_parameter_layer2)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer3_2", adaptive_parameter_layer3)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer4_2", adaptive_parameter_layer4)
    np.save(f"../results/{folder}/{ID}/adaptive_parameter_layer5_2", adaptive_parameter_layer5)
    
    # store data

    np.save(f"../results/{folder}/{ID}/theta_pred", theta_pred)
    np.save(f"../results/{folder}/{ID}/psi_pred", psi_pred)
    np.save(f"../results/{folder}/{ID}/K_pred", K_pred)
    np.save(f"../results/{folder}/{ID}/flux_pred", flux_pred)
    np.save(f"../results/{folder}/{ID}/residual", residual)

    np.save(f"../results/{folder}/{ID}/loss_ic_1", loss_ic_1)
    np.save(f"../results/{folder}/{ID}/loss_ic_1_BFGS", loss_ic_1_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_ic_2", loss_ic_1)
    np.save(f"../results/{folder}/{ID}/loss_ic_2_BFGS", loss_ic_2_BFGS)

    np.save(f"../results/{folder}/{ID}/loss_ub", loss_ub)
    np.save(f"../results/{folder}/{ID}/loss_ub_BFGS", loss_ub_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_lb", loss_lb)
    np.save(f"../results/{folder}/{ID}/loss_lb_BFGS", loss_lb_BFGS)

    np.save(f"../results/{folder}/{ID}/loss_res_1", loss_res_1)
    np.save(f"../results/{folder}/{ID}/loss_res_1_BFGS", loss_res_1_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_res_2", loss_res_2)
    np.save(f"../results/{folder}/{ID}/loss_res_2_BFGS", loss_res_2_BFGS)

    np.save(f"../results/{folder}/{ID}/loss_psi_int", loss_psi_int)
    np.save(f"../results/{folder}/{ID}/loss_psi_int_BFGS", loss_psi_int_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_qz_int", loss_qz_int)
    np.save(f"../results/{folder}/{ID}/loss_qz_int_BFGS", loss_qz_int_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_res_int", loss_res_int)
    np.save(f"../results/{folder}/{ID}/loss_res_int_BFGS", loss_res_int_BFGS)
    
    


    np.save(f"../results/{folder}/{ID}/t_res", t_res)
    np.save(f"../results/{folder}/{ID}/z_res", z_res)
    if RAR:
        np.save(f"../results/{folder}/{ID}/new_residual", new_residual)

    np.save(f"../results/{folder}/{ID}/z_mesh", z_mesh)
    np.save(f"../results/{folder}/{ID}/t_mesh", t_mesh)


    sheet1.write(ID, 0, ID)
    sheet1.write(ID, 1, random_seed)
    sheet1.write(ID, 2, number_layers_1)
    sheet1.write(ID, 3, number_units_1)
    sheet1.write(ID, 4, number_layers_2)
    sheet1.write(ID, 5, number_units_2)
    sheet1.write(ID, 6, LAA)
    sheet1.write(ID, 7, number_res_1)
    sheet1.write(ID, 8, number_res_2)
    sheet1.write(ID, 9, number_ub)
    sheet1.write(ID, 10, number_int)

    sheet1.write(ID, 11, weight_values[0])
    sheet1.write(ID, 12, weight_values[1])
    sheet1.write(ID, 13, weight_values[2])
    sheet1.write(ID, 14, weight_values[3])
    sheet1.write(ID, 15, weight_values[4])
    sheet1.write(ID, 16, weight_values[5])
    sheet1.write(ID, 17, weight_values[6])
    sheet1.write(ID, 18, weight_values[7])

    sheet1.write(ID, 19, ALR)
    sheet1.write(ID, 20, iterations)
    sheet1.write(ID, 21, batch_size)
    sheet1.write(ID, 22, RAR)
    sheet1.write(ID, 23, new_residual_points)
    sheet1.write(ID, 24, time_Adam)
    sheet1.write(ID, 25, time_BFGS)

    sheet1.write(ID, 26, theta_L2_relative_error)
    sheet1.write(ID, 27, theta_abs_relative_error)
    sheet1.write(ID, 28, theta_max_error)
    sheet1.write(ID, 29, psi_L2_relative_error)
    sheet1.write(ID, 30, psi_abs_relative_error)
    sheet1.write(ID, 31, psi_max_error)
    sheet1.write(ID, 32, str(loss_ub_BFGS[-1]))
    sheet1.write(ID, 33, str(loss_lb_BFGS[-1]))
    sheet1.write(ID, 34, str(loss_ic_1_BFGS[-1]))
    sheet1.write(ID, 35, str(loss_ic_2_BFGS[-1]))
    sheet1.write(ID, 36, str(loss_res_1_BFGS[-1]))
    sheet1.write(ID, 37, str(loss_res_2_BFGS[-1]))
    sheet1.write(ID, 38, str(loss_res_int_BFGS[-1]))
    sheet1.write(ID, 39, str(loss_psi_int_BFGS[-1]))
    sheet1.write(ID, 40, str(loss_qz_int_BFGS[-1]))


In [ ]:
folder = "example-heterogeneous"
wb = xlwt.Workbook()
sheet1 = wb.add_sheet(f'{folder}')
sheet1.write(0, 0,"ID")
sheet1.write(0, 1, "random_seed")
sheet1.write(0, 2, "number_layers_1")
sheet1.write(0, 3, "number_units_1")
sheet1.write(0, 4, "number_layers_2")
sheet1.write(0, 5, "number_units_2")
sheet1.write(0, 6, "LAA")
sheet1.write(0, 7, "number_res_1")
sheet1.write(0, 8, "number_res_2")
sheet1.write(0, 9, "number_ub")
sheet1.write(0, 10, "number_int")
sheet1.write(0, 11, "weight_values_ic_1")
sheet1.write(0, 12, "weight_values_lb")
sheet1.write(0, 13, "weight_values_res_2")
sheet1.write(0, 14, "weight_values_ic_2")
sheet1.write(0, 15, "weight_values_ub")
sheet1.write(0, 16, "weight_values_int") # this should have been res_int
sheet1.write(0, 17, "weight_values_psi_int")
sheet1.write(0, 18, "weight_values_qz_int")
sheet1.write(0, 19, "ALR")
sheet1.write(0, 20, "iterations")
sheet1.write(0, 21, "batch_size")
sheet1.write(0, 22, "RAR")
sheet1.write(0, 23, "new_residual_points")
sheet1.write(0, 24, "time_Adam")
sheet1.write(0, 25, "time_BFGS")
sheet1.write(0, 26, "theta_L2_relative_error")
sheet1.write(0, 27, "theta_abs_relative_error")
sheet1.write(0, 28, "theta_max_error")
sheet1.write(0, 29, "psi_L2_relative_error")
sheet1.write(0, 30, "psi_abs_relative_error")
sheet1.write(0, 31, "psi_max_error")
sheet1.write(0, 32,  "loss_ub")
sheet1.write(0, 33,  "loss_lb")
sheet1.write(0, 34,  "loss_ic_1")
sheet1.write(0, 35,  "loss_ic_2")
sheet1.write(0, 36,  "loss_res_1")
sheet1.write(0, 37,  "loss_res_2")
sheet1.write(0, 38,  "loss_res_int")
sheet1.write(0, 39,  "loss_psi_int")
sheet1.write(0, 40,  "loss_qz_int")

random_seed = 1
iteration_list = [100000]
ID = 1 # must be larger than 0
for i in range(len(iteration_list)):

    main_loop(folder, ID, random_seed, iteration_list[i])
    ID += 1
wb.save(f'../results/{folder}/summary.xls')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5

It: 0, Loss_res_1: 3.334e-03, Loss_ic_1: 1.464e-01, Loss_lb: 1.204e-02,                                 Loss_res_2: 1.978e-01, Loss_ic_2: 5.327e-02, Loss_ub: 3.415e+01,                                 Loss_psi_int: 5.646e-02, Loss_qz_int: 7.395e+01, Loss_res_int: 7.675e-01, Time: 4.31
It: 10, Loss_res_1: 1.483e-02, Loss_ic_1: 2.282e-02, Loss_lb: 1.371e-02,                                 Loss_res_2: 1.002e-02, Loss_ic_2: 2.402e-03, Loss_ub: 6.682e-01,                        

It: 270, Loss_res_1: 1.356e-02, Loss_ic_1: 5.357e-03, Loss_lb: 3.750e-03,                                 Loss_res_2: 8.677e-03, Loss_ic_2: 1.711e-03, Loss_ub: 9.409e-04,                                 Loss_psi_int: 1.272e-03, Loss_qz_int: 4.138e-03, Loss_res_int: 6.374e-03, Time: 0.54
It: 280, Loss_res_1: 1.274e-02, Loss_ic_1: 4.841e-03, Loss_lb: 3.665e-03,                                 Loss_res_2: 9.097e-03, Loss_ic_2: 1.675e-03, Loss_ub: 8.955e-04,                                 Loss_psi_int: 1.165e-03, Loss_qz_int: 3.837e-03, Loss_res_int: 5.787e-03, Time: 0.52
It: 290, Loss_res_1: 1.339e-02, Loss_ic_1: 4.451e-03, Loss_lb: 3.552e-03,                                 Loss_res_2: 1.099e-02, Loss_ic_2: 1.640e-03, Loss_ub: 9.053e-04,                                 Loss_psi_int: 1.100e-03, Loss_qz_int: 3.548e-03, Loss_res_int: 5.262e-03, Time: 0.51
It: 300, Loss_res_1: 1.408e-02, Loss_ic_1: 4.128e-03, Loss_lb: 3.430e-03,                                 Loss_res_2: 1.455e-02, Loss_ic

It: 560, Loss_res_1: 1.698e-02, Loss_ic_1: 1.624e-03, Loss_lb: 1.691e-03,                                 Loss_res_2: 9.401e-03, Loss_ic_2: 1.090e-03, Loss_ub: 5.467e-04,                                 Loss_psi_int: 4.639e-04, Loss_qz_int: 1.068e-03, Loss_res_int: 8.170e-04, Time: 0.51
It: 570, Loss_res_1: 1.479e-02, Loss_ic_1: 1.583e-03, Loss_lb: 1.681e-03,                                 Loss_res_2: 1.197e-02, Loss_ic_2: 1.076e-03, Loss_ub: 5.209e-04,                                 Loss_psi_int: 4.574e-04, Loss_qz_int: 1.046e-03, Loss_res_int: 7.784e-04, Time: 0.50
It: 580, Loss_res_1: 1.623e-02, Loss_ic_1: 1.578e-03, Loss_lb: 1.629e-03,                                 Loss_res_2: 1.171e-02, Loss_ic_2: 1.071e-03, Loss_ub: 4.707e-04,                                 Loss_psi_int: 4.538e-04, Loss_qz_int: 1.027e-03, Loss_res_int: 7.540e-04, Time: 0.51
It: 590, Loss_res_1: 1.479e-02, Loss_ic_1: 1.554e-03, Loss_lb: 1.605e-03,                                 Loss_res_2: 1.186e-02, Loss_ic

It: 850, Loss_res_1: 1.410e-02, Loss_ic_1: 1.207e-03, Loss_lb: 1.105e-03,                                 Loss_res_2: 9.947e-03, Loss_ic_2: 7.994e-04, Loss_ub: 2.934e-04,                                 Loss_psi_int: 3.911e-04, Loss_qz_int: 7.623e-04, Loss_res_int: 3.048e-04, Time: 0.50
It: 860, Loss_res_1: 1.781e-02, Loss_ic_1: 1.187e-03, Loss_lb: 1.105e-03,                                 Loss_res_2: 9.830e-03, Loss_ic_2: 7.959e-04, Loss_ub: 2.976e-04,                                 Loss_psi_int: 3.876e-04, Loss_qz_int: 7.603e-04, Loss_res_int: 3.043e-04, Time: 0.55
It: 870, Loss_res_1: 1.736e-02, Loss_ic_1: 1.192e-03, Loss_lb: 1.071e-03,                                 Loss_res_2: 9.178e-03, Loss_ic_2: 7.869e-04, Loss_ub: 3.149e-04,                                 Loss_psi_int: 3.864e-04, Loss_qz_int: 7.567e-04, Loss_res_int: 3.010e-04, Time: 0.51
It: 880, Loss_res_1: 1.407e-02, Loss_ic_1: 1.178e-03, Loss_lb: 1.066e-03,                                 Loss_res_2: 1.050e-02, Loss_ic

It: 1140, Loss_res_1: 1.770e-02, Loss_ic_1: 9.878e-04, Loss_lb: 7.435e-04,                                 Loss_res_2: 9.749e-03, Loss_ic_2: 6.077e-04, Loss_ub: 2.053e-04,                                 Loss_psi_int: 3.191e-04, Loss_qz_int: 6.839e-04, Loss_res_int: 1.619e-04, Time: 0.52
It: 1150, Loss_res_1: 1.673e-02, Loss_ic_1: 9.808e-04, Loss_lb: 7.333e-04,                                 Loss_res_2: 9.424e-03, Loss_ic_2: 6.026e-04, Loss_ub: 1.618e-04,                                 Loss_psi_int: 3.165e-04, Loss_qz_int: 6.822e-04, Loss_res_int: 1.566e-04, Time: 0.51
It: 1160, Loss_res_1: 1.714e-02, Loss_ic_1: 9.827e-04, Loss_lb: 7.096e-04,                                 Loss_res_2: 9.677e-03, Loss_ic_2: 5.984e-04, Loss_ub: 1.656e-04,                                 Loss_psi_int: 3.135e-04, Loss_qz_int: 6.823e-04, Loss_res_int: 1.548e-04, Time: 0.50
It: 1170, Loss_res_1: 1.465e-02, Loss_ic_1: 9.723e-04, Loss_lb: 7.059e-04,                                 Loss_res_2: 8.974e-03, Los

...

It: 18250, Loss_res_1: 8.869e-04, Loss_ic_1: 1.842e-06, Loss_lb: 2.373e-06,                                 Loss_res_2: 2.979e-04, Loss_ic_2: 2.022e-04, Loss_ub: 2.097e-05,                                 Loss_psi_int: 2.180e-05, Loss_qz_int: 1.762e-04, Loss_res_int: 2.180e-04, Time: 0.54
It: 18260, Loss_res_1: 3.833e-04, Loss_ic_1: 2.003e-06, Loss_lb: 5.424e-07,                                 Loss_res_2: 2.713e-04, Loss_ic_2: 2.036e-04, Loss_ub: 9.053e-06,                                 Loss_psi_int: 4.085e-06, Loss_qz_int: 1.367e-05, Loss_res_int: 6.799e-06, Time: 0.52
It: 18270, Loss_res_1: 5.108e-04, Loss_ic_1: 2.386e-06, Loss_lb: 2.208e-06,                                 Loss_res_2: 2.836e-04, Loss_ic_2: 2.026e-04, Loss_ub: 1.298e-05,                                 Loss_psi_int: 4.303e-06, Loss_qz_int: 7.195e-05, Loss_res_int: 1.907e-04, Time: 0.50
It: 18280, Loss_res_1: 8.985e-04, Loss_ic_1: 2.243e-06, Loss_lb: 2.096e-06,                                 Loss_res_2: 2.274e-04,

It: 18540, Loss_res_1: 4.151e-04, Loss_ic_1: 1.837e-06, Loss_lb: 6.348e-07,                                 Loss_res_2: 2.397e-04, Loss_ic_2: 2.037e-04, Loss_ub: 1.074e-05,                                 Loss_psi_int: 3.661e-06, Loss_qz_int: 9.984e-05, Loss_res_int: 4.556e-05, Time: 0.50
It: 18550, Loss_res_1: 4.944e-04, Loss_ic_1: 1.774e-06, Loss_lb: 4.757e-07,                                 Loss_res_2: 3.737e-04, Loss_ic_2: 2.013e-04, Loss_ub: 2.414e-05,                                 Loss_psi_int: 1.449e-05, Loss_qz_int: 1.097e-04, Loss_res_int: 8.425e-05, Time: 0.53
It: 18560, Loss_res_1: 5.168e-04, Loss_ic_1: 1.780e-06, Loss_lb: 6.668e-07,                                 Loss_res_2: 2.269e-04, Loss_ic_2: 2.035e-04, Loss_ub: 1.083e-05,                                 Loss_psi_int: 5.446e-06, Loss_qz_int: 2.117e-04, Loss_res_int: 2.164e-04, Time: 0.49
It: 18570, Loss_res_1: 2.644e-04, Loss_ic_1: 1.876e-06, Loss_lb: 5.014e-07,                                 Loss_res_2: 1.994e-04,

It: 18830, Loss_res_1: 4.754e-04, Loss_ic_1: 2.080e-06, Loss_lb: 1.056e-06,                                 Loss_res_2: 1.495e-04, Loss_ic_2: 2.009e-04, Loss_ub: 9.175e-06,                                 Loss_psi_int: 4.062e-06, Loss_qz_int: 9.319e-05, Loss_res_int: 1.291e-04, Time: 0.52
It: 18840, Loss_res_1: 5.813e-04, Loss_ic_1: 1.501e-06, Loss_lb: 5.936e-07,                                 Loss_res_2: 2.781e-04, Loss_ic_2: 2.010e-04, Loss_ub: 7.691e-06,                                 Loss_psi_int: 3.598e-06, Loss_qz_int: 3.660e-05, Loss_res_int: 9.568e-06, Time: 0.49
It: 18850, Loss_res_1: 3.952e-04, Loss_ic_1: 1.535e-06, Loss_lb: 4.780e-07,                                 Loss_res_2: 2.395e-04, Loss_ic_2: 1.998e-04, Loss_ub: 1.219e-05,                                 Loss_psi_int: 6.952e-06, Loss_qz_int: 1.911e-05, Loss_res_int: 3.409e-05, Time: 0.52
It: 18860, Loss_res_1: 4.660e-04, Loss_ic_1: 2.049e-06, Loss_lb: 1.526e-06,                                 Loss_res_2: 1.979e-04,

It: 19120, Loss_res_1: 6.955e-04, Loss_ic_1: 1.722e-06, Loss_lb: 1.486e-06,                                 Loss_res_2: 2.656e-04, Loss_ic_2: 2.005e-04, Loss_ub: 1.511e-05,                                 Loss_psi_int: 1.041e-05, Loss_qz_int: 3.400e-04, Loss_res_int: 2.615e-04, Time: 0.51
It: 19130, Loss_res_1: 5.751e-04, Loss_ic_1: 1.864e-06, Loss_lb: 5.533e-07,                                 Loss_res_2: 1.573e-04, Loss_ic_2: 1.992e-04, Loss_ub: 1.136e-05,                                 Loss_psi_int: 5.122e-06, Loss_qz_int: 1.339e-04, Loss_res_int: 1.047e-04, Time: 0.54
It: 19140, Loss_res_1: 5.131e-04, Loss_ic_1: 1.473e-06, Loss_lb: 1.102e-06,                                 Loss_res_2: 2.100e-04, Loss_ic_2: 1.983e-04, Loss_ub: 9.422e-06,                                 Loss_psi_int: 3.524e-06, Loss_qz_int: 7.004e-05, Loss_res_int: 6.868e-05, Time: 0.53
It: 19150, Loss_res_1: 3.466e-04, Loss_ic_1: 1.392e-06, Loss_lb: 5.592e-07,                                 Loss_res_2: 3.313e-04,

It: 19410, Loss_res_1: 4.729e-04, Loss_ic_1: 1.530e-06, Loss_lb: 1.379e-06,                                 Loss_res_2: 2.247e-04, Loss_ic_2: 2.008e-04, Loss_ub: 4.365e-05,                                 Loss_psi_int: 2.123e-05, Loss_qz_int: 1.502e-03, Loss_res_int: 1.478e-03, Time: 0.53
It: 19420, Loss_res_1: 5.108e-04, Loss_ic_1: 1.892e-06, Loss_lb: 6.196e-07,                                 Loss_res_2: 1.763e-04, Loss_ic_2: 1.977e-04, Loss_ub: 1.252e-05,                                 Loss_psi_int: 7.272e-06, Loss_qz_int: 3.070e-04, Loss_res_int: 1.819e-04, Time: 0.51
It: 19430, Loss_res_1: 3.440e-04, Loss_ic_1: 1.786e-06, Loss_lb: 7.475e-07,                                 Loss_res_2: 2.480e-04, Loss_ic_2: 1.965e-04, Loss_ub: 9.856e-06,                                 Loss_psi_int: 4.001e-06, Loss_qz_int: 1.344e-05, Loss_res_int: 3.301e-05, Time: 0.51
It: 19440, Loss_res_1: 3.155e-04, Loss_ic_1: 1.514e-06, Loss_lb: 4.734e-07,                                 Loss_res_2: 1.605e-04,